In [6]:
import argparse
import os
import random
from torch.autograd import Variable
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data
import torchvision
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML
from matplotlib.pyplot import imshow, imsave

# Set random seem for reproducibility
manualSeed = 1312
#manualSeed = random.randint(1, 10000) # use if you want new results
print("Random Seed: ", manualSeed)
random.seed(manualSeed)
torch.manual_seed(manualSeed)

print(torchvision.__version__)
# This is used to check if there are corrupted image files.

class ImageFolderWithPaths(datasets.ImageFolder):
    """Custom dataset that includes image file paths. Extends
    torchvision.datasets.ImageFolder
    """

    # override the __getitem__ method. this is the method dataloader calls
    def __getitem__(self, index):
        # this is what ImageFolder normally returns 
        original_tuple = super(ImageFolderWithPaths, self).__getitem__(index)
        # the image file path
        path = self.imgs[index][0]
        # make a new tuple that includes original and the path
        tuple_with_path = (original_tuple + (path,))
        return tuple_with_path


# instantiate the dataset and dataloader
transform=torchvision.transforms.Compose([
                                torchvision.transforms.Resize(64),
                                torchvision.transforms.ToTensor(),
                                torchvision.transforms.Normalize(
                                 (0.5,0.5,0.5), (0.5,0.5,0.5))])

data_dir = F"./Dog_images/Label5/"
dataset = ImageFolderWithPaths(data_dir, transform = transform)
print(dataset)# our custom dataset
dataloader = torch.utils.data.DataLoader(dataset)

# iterate over data
for inputs, labels, paths in dataloader:
    # use the above variables freely
    print(paths)

Random Seed:  1312
0.2.1
Dataset ImageFolderWithPaths
    Number of datapoints: 6600
    Root Location: ./Dog_images/Label5/
    Transforms (if any): Compose(
                             Resize(size=64, interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None
('./Dog_images/Label5/cityscape\\1-The Ghent Altarpiece (detail).jpg',)
('./Dog_images/Label5/cityscape\\10-The Trefiler&#237;a on Peignitz.jpg',)
('./Dog_images/Label5/cityscape\\100-Lagoon landscape.jpg',)
('./Dog_images/Label5/cityscape\\1000-London Bridge on the Night of the Marriage of the Prince and Princess of Wales.jpg',)
('./Dog_images/Label5/cityscape\\1001-Street of Marlotte (also known as Women Going to the Woods).jpg',)
('./Dog_images/Label5/cityscape\\1002-A Square in La Roche Guyon.jpg',)
('./Dog_images/Label5/cityscape\\1003-Saint Germain l&#39;Auxerrois

('./Dog_images/Label5/cityscape\\1095-Zaandam.jpg',)
('./Dog_images/Label5/cityscape\\1096-Houses on the Achterzaan.jpg',)
('./Dog_images/Label5/cityscape\\1097-At the Sink.jpg',)
('./Dog_images/Label5/cityscape\\1098-Heidelberg Castle, Germany.jpg',)
('./Dog_images/Label5/cityscape\\1099-Seville, Spain.jpg',)
('./Dog_images/Label5/cityscape\\11-Wire Drawing Mill.jpg',)
('./Dog_images/Label5/cityscape\\110-Rome_ View of the Piazza di San Giovanni in Laterano.jpg',)
('./Dog_images/Label5/cityscape\\1100-A Home on the Mississippi.jpg',)
('./Dog_images/Label5/cityscape\\1101-The Crystal Palace, London.jpg',)
('./Dog_images/Label5/cityscape\\1102-Golden Horn.jpg',)
('./Dog_images/Label5/cityscape\\1103-The Bay Golden Horn in Istanbul.jpg',)
('./Dog_images/Label5/cityscape\\1104-The Seine at Port Marly.jpg',)
('./Dog_images/Label5/cityscape\\1105-Argenteuil, the Hospice.jpg',)
('./Dog_images/Label5/cityscape\\1106-Carrieres-Saint-Denis.jpg',)
('./Dog_images/Label5/cityscape\\1107-Seine at R

('./Dog_images/Label5/cityscape\\1194-The House and the Tree.jpg',)
('./Dog_images/Label5/cityscape\\1195-The Seine at the Pont d&#39;Iena.jpg',)
('./Dog_images/Label5/cityscape\\1196-The Grand Canal.jpg',)
('./Dog_images/Label5/cityscape\\1197-Lunar night at the sea.jpg',)
('./Dog_images/Label5/cityscape\\1198-Boulevard Saint-Denis, Argenteuil, in Winter.jpg',)
('./Dog_images/Label5/cityscape\\1199-Snow Effect with Setting Sun.jpg',)
('./Dog_images/Label5/cityscape\\12-Courtyard of the Former Castle in Innsbruck without Clouds.jpg',)
('./Dog_images/Label5/cityscape\\120-View of Verona and the River Adige from the Ponte Nuovo.jpg',)
('./Dog_images/Label5/cityscape\\1200-Snow Effect, The Boulevard de Pontoise at Argenteuil.jpg',)
('./Dog_images/Label5/cityscape\\1201-Snow Effect, The Street in Argentuil.jpg',)
('./Dog_images/Label5/cityscape\\1202-View of Argenteuil in the Snow.jpg',)
('./Dog_images/Label5/cityscape\\1203-Place de la Trinite.jpg',)
('./Dog_images/Label5/cityscape\\1204-

('./Dog_images/Label5/cityscape\\1301-Duane Street, New York.jpg',)
('./Dog_images/Label5/cityscape\\1302-The Pont de l&#39;Europe, Gare Saint-Lazare.jpg',)
('./Dog_images/Label5/cityscape\\1303-Funchal, Morning Sun.jpg',)
('./Dog_images/Label5/cityscape\\1304-Moscow court.jpg',)
('./Dog_images/Label5/cityscape\\1305-Orchard in Pontoise.jpg',)
('./Dog_images/Label5/cityscape\\1306-In a Southern City.jpg',)
('./Dog_images/Label5/cityscape\\1307-View Across a Creek.jpg',)
('./Dog_images/Label5/cityscape\\1308-Wash Day.jpg',)
('./Dog_images/Label5/cityscape\\1309-Market Day.jpg',)
('./Dog_images/Label5/cityscape\\131-The New Market Square in Dresden.jpg',)
('./Dog_images/Label5/cityscape\\1310-Charleston.jpg',)
('./Dog_images/Label5/cityscape\\1311-Red Roofs.jpg',)
('./Dog_images/Label5/cityscape\\1312-The road-menders, Rue de Berne.jpg',)
('./Dog_images/Label5/cityscape\\1313-Ka d&#39;Ordo Palace in Venice by moonlight.jpg',)
('./Dog_images/Label5/cityscape\\1314-Palace rains in Venice b

('./Dog_images/Label5/cityscape\\1415-Near the Harbor.jpg',)
('./Dog_images/Label5/cityscape\\1416-Lincoln Cathedral, the Cloisters.jpg',)
('./Dog_images/Label5/cityscape\\1417-Street Scene, Cairo (also known as A North African Street).jpg',)
('./Dog_images/Label5/cityscape\\1418-The Mosque of Emir Mindar in Cairo.jpg',)
('./Dog_images/Label5/cityscape\\1419-Clovelly.jpg',)
('./Dog_images/Label5/cityscape\\142-Courtyard of the Castle at K&#246;nigstein from the South.jpg',)
('./Dog_images/Label5/cityscape\\1420-Street Scene, Cairo.jpg',)
('./Dog_images/Label5/cityscape\\1421-Street Scene, Cairo.jpg',)
('./Dog_images/Label5/cityscape\\1422-The Coptic Quarter, Cairo.jpg',)
('./Dog_images/Label5/cityscape\\1423-The Mosque of Khair Bek, Cairo.jpg',)
('./Dog_images/Label5/cityscape\\1424-View to Michael&#39;s Castle in Petersburg from Lebiazhy Canal.jpg',)
('./Dog_images/Label5/cityscape\\1425-Reflections on the Thames, Westminster.jpg',)
('./Dog_images/Label5/cityscape\\1426-Old Aker Churc

...

('./Dog_images/Label5/cityscape\\649-Rome, Castle Sant Angelo.jpg',)
('./Dog_images/Label5/cityscape\\6490-L&#39;Arc de Triomphe.jpg',)
('./Dog_images/Label5/cityscape\\6491-Le boulevard Paris.jpg',)
('./Dog_images/Label5/cityscape\\6492-Paris, La Chatelet.jpg',)
('./Dog_images/Label5/cityscape\\6493-Paris, Rue Tronchet.jpg',)
('./Dog_images/Label5/cityscape\\6494-Place Vendome.jpg',)
('./Dog_images/Label5/cityscape\\6495-Porte St. Martin.jpg',)
('./Dog_images/Label5/cityscape\\6496-Porte St. Denis.jpg',)
('./Dog_images/Label5/cityscape\\6497-Porte St. Martin.jpg',)
('./Dog_images/Label5/cityscape\\6498-Porte St. Martin.jpg',)
('./Dog_images/Label5/cityscape\\6499-Porte St. Martin.jpg',)
('./Dog_images/Label5/cityscape\\65-Cityscape with Drawbridge.jpg',)
('./Dog_images/Label5/cityscape\\650-Rome, the Basilica of Constantine.jpg',)
('./Dog_images/Label5/cityscape\\6500-Arc de Triomphe.jpg',)
('./Dog_images/Label5/cityscape\\6501-Arc de Triomphe.jpg',)
('./Dog_images/Label5/cityscape\\6

('./Dog_images/Label5/cityscape\\667-A View of the Pavillon de Flore and the Tuileries from the Seine, Notre Dame, Paris.jpg',)
('./Dog_images/Label5/cityscape\\668-Pont Neuf from the Quai de l&#39;Ecole, Paris.jpg',)
('./Dog_images/Label5/cityscape\\669-Near the Pont d&#39;Arcole, Paris.jpg',)
('./Dog_images/Label5/cityscape\\67-View in Amsterdam.jpg',)
('./Dog_images/Label5/cityscape\\670-Place St. Barth&#233;lemy, Rouen.jpg',)
('./Dog_images/Label5/cityscape\\671-Porte St. Denis, Paris.jpg',)
('./Dog_images/Label5/cityscape\\672-St. Eustache, Paris.jpg',)
('./Dog_images/Label5/cityscape\\673-St. Eustache, Paris.jpg',)
('./Dog_images/Label5/cityscape\\674-Tour d&#39;Horloge, Rouen.jpg',)
('./Dog_images/Label5/cityscape\\675-Chartres Cathedral.jpg',)
('./Dog_images/Label5/cityscape\\676-Fishing Boats Tied to the Wharf.jpg',)
('./Dog_images/Label5/cityscape\\677-The Inn at Montigny les Cormeilles.jpg',)
('./Dog_images/Label5/cityscape\\678-View of the Pont au Change from Quai de Gesvre

('./Dog_images/Label5/cityscape\\764-The dream of the architect.jpg',)
('./Dog_images/Label5/cityscape\\765-South of Bellinzona.jpg',)
('./Dog_images/Label5/cityscape\\766-Ruins of Diocletian at Spalato.jpg',)
('./Dog_images/Label5/cityscape\\767-A View of a Town along the Rhine.jpg',)
('./Dog_images/Label5/cityscape\\768-Hafod.jpg',)
('./Dog_images/Label5/cityscape\\769-Amalfi.jpg',)
('./Dog_images/Label5/cityscape\\77-Entrance To The Grand Canal Looking East.jpg',)
('./Dog_images/Label5/cityscape\\770-The Fountains of Trevi.jpg',)
('./Dog_images/Label5/cityscape\\771-Departure of an Omnibus from St. Isaac&#39;s Square in St. Petersburg.jpg',)
('./Dog_images/Label5/cityscape\\772-Giudecca, la Donna della Salute and San Georgio.jpg',)
('./Dog_images/Label5/cityscape\\773-Sorrento. Sea view.jpg',)
('./Dog_images/Label5/cityscape\\774-Venice.jpg',)
('./Dog_images/Label5/cityscape\\775-Venice.jpg',)
('./Dog_images/Label5/cityscape\\776-Campo Santo, Venice.jpg',)
('./Dog_images/Label5/city

('./Dog_images/Label5/cityscape\\865-The South Side of St Mark.jpg',)
('./Dog_images/Label5/cityscape\\866-Smolny as seen from Bolshaya Okhta.jpg',)
('./Dog_images/Label5/cityscape\\867-View of the Church of Santa Maria della Salute, on the Grand Canal, Venice, with the Dogana beyond.jpg',)
('./Dog_images/Label5/cityscape\\868-A Canal Tunnel Near Leeds.jpg',)
('./Dog_images/Label5/cityscape\\869-Ivy Bridge, Devonshire.jpg',)
('./Dog_images/Label5/cityscape\\87-The Riva degli Schiavoni.jpg',)
('./Dog_images/Label5/cityscape\\870-Kidwelly Castle.jpg',)
('./Dog_images/Label5/cityscape\\871-Melrose Abbey.jpg',)
('./Dog_images/Label5/cityscape\\872-Old London Brige.jpg',)
('./Dog_images/Label5/cityscape\\873-Rome From Monte Testaccio.jpg',)
('./Dog_images/Label5/cityscape\\874-Stamford.jpg',)
('./Dog_images/Label5/cityscape\\875-Venice.jpg',)
('./Dog_images/Label5/cityscape\\876-The Grand Canal Scene, A Street In Venice.jpg',)
('./Dog_images/Label5/cityscape\\877-Valley of The Brook Kedron.

('./Dog_images/Label5/cityscape\\971-A visit of Bayliff Ten Frootenhuys to the Guild of Archers.jpg',)
('./Dog_images/Label5/cityscape\\972-Architecture and Art of the great Temple of Karnak. City of Thebes. Egypt..jpg',)
('./Dog_images/Label5/cityscape\\973-Pelra. Views of the Eastern End of the Valley.jpg',)
('./Dog_images/Label5/cityscape\\974-Baalbek. Remains of the Western Portico of the Temple of Jupiter.jpg',)
('./Dog_images/Label5/cityscape\\975-Bethlehem. Shrine of the Nativity.jpg',)
('./Dog_images/Label5/cityscape\\976-Jerusalem from the Mount of Olives.jpg',)
('./Dog_images/Label5/cityscape\\977-Baalbek. The Circular Temple.jpg',)
('./Dog_images/Label5/cityscape\\978-Portion of the Eastern Portico.jpg',)
('./Dog_images/Label5/cityscape\\979-Entrance to the Citadel.jpg',)
('./Dog_images/Label5/cityscape\\98-Padua_ The Prato della Valle with Santa Giustinia and the Church of Misericordia.jpg',)
('./Dog_images/Label5/cityscape\\980-Jerusalem. The Damascus Gate.jpg',)
('./Dog_i